In [1]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
def denoise(X, y, n_neighbors = 3):
    """
    Given a set of predictors and targets, determines which class is the minority and then looks for members
    of the minority class for which the closest neighbors are all in the majority. Those elements will be
    pruned from the set.
    """
    # Identify the classes and how many of each exist.
    classes, class_counts = np.unique(y, return_counts = True)
    print(f"Classes: {classes}")

    # Determine which class has the lowest representation in the dataset.
    minority_index = np.argmin(class_counts)
    minority_class = classes[minority_index]
    print(f"Minority Class: {minority_class}")
    print(f"Minority Ratio: {class_counts[minority_index] / y.size}")

    # Build a model representing the nearest neighbors and set up a list to cache the points that are noise.
    knn = NearestNeighbors(n_neighbors = n_neighbors + 1).fit(X)
    noise = []
    
    # Identify which data points represent the minority.
    minority_indices = np.where(y == minority_class)

    # Get the indices of the closest neighbors for each minority sample and then loop over the collection.
    ind = knn.kneighbors(X[minority_indices], return_distance = False)
    for test_index in np.arange(len(ind)):
        # Ignore the closest neighbor because that's the point itself.
        neighbor_indices = ind[test_index][1:]

        # Check for all of the others to _not_ be in the minority class and add to the noise list if that's the case.
        if np.all([y[i] != minority_class for i in neighbor_indices]):
            noise.append(test_index)

    print(f"Noisy samples: {np.array(noise)}")
    return np.delete(X, noise, axis = 0), np.delete(y, noise, axis = 0)

In [3]:
def get_category_indices(X, y, n_neighbors = 3):
    """
    Determine and return the sets of indices representing the minority, the boundary majority, and the remaining majority.
    """
    # Identify the classes and how many of each exist.
    classes, class_counts = np.unique(y, return_counts = True)
    print(f"Classes: {classes}")

    # Determine which class has the lowest representation in the dataset.
    minority_index = np.argmin(class_counts)
    minority_class = classes[minority_index]
    print(f"Minority Class: {minority_class}")
    print(f"Minority Ratio: {class_counts[minority_index] / y.size}")

    # Build a model representing the nearest neighbors and set up a list to cache the points that are noise.
    knn = NearestNeighbors(n_neighbors = n_neighbors + 1).fit(X)

    # Get the minority indices ... that's the easy one.
    minority_indices = np.where(y == minority_class)
    
    bmaj = []
    obmaj = []
    majority_indices = np.where(y != minority_class)

    # Get the indices of the closest neighbors for each majority sample and then loop over the collection.
    ind = knn.kneighbors(X[majority_indices], return_distance = False)
    for test_index in np.arange(len(ind)):
        # Ignore the closest neighbor because that's the point itself.
        neighbor_indices = ind[test_index][1:]

        if np.any([y[i] == minority_class for i in neighbor_indices]):
            bmaj.append(test_index)
        else:
            obmaj.append(test_index)

    return minority_indices[0], np.array(bmaj), np.array(obmaj)

In [4]:
def sort_majority_samples(X):
    """
    Given a set of samples, return a list of indices into that collection sorted by descending distance to closest neighbor.
    """
    p = len(X[0])
    n = len(X)
    mat = np.zeros((n, n))
    for row in np.arange(n):
        for col in np.arange(n):
            if col == row:
                continue
            sum = np.zeros((p, p))
            for i in np.arange(n):
                q = X[i] - X[col]
                sum += np.outer(q, q)
            S_j = (1.0 / (n - 1)) * sum
            x = X[row] - X[col]
            mat[row, col] = (1.0 / p) * np.abs(np.dot(np.dot(x, np.linalg.inv(S_j)), x))

    # Now the distance between observations i and j is the minimum of mat[i,j] and mat[j,i]
    # For each entry we want to find the distance to its closest neighbor.
    pairs = []
    for obs in np.arange(n):
        row_vals = np.delete(mat[obs, :], obs)
        col_vals = np.delete(mat[:, obs], obs)
        pairs.append([obs, np.min(np.append(row_vals, col_vals))])
    nearest = np.array(pairs)
    sorted_indices = nearest[:, 1].argsort()[::-1]
    return sorted_indices

In [5]:
from sklearn.datasets import make_classification
X, y = make_classification(n_classes=2, class_sep=2, weights=[0.9, 0.1], n_informative=3, n_redundant=1,
                           flip_y=0, n_features=20, n_clusters_per_class=1, n_samples=100, random_state=10)
min_val = np.min(X)
if min_val < 0:
    X += abs(min_val)

In [6]:
X_new, y_new = denoise(X, y)

Classes: [0 1]
Minority Class: 1
Minority Ratio: 0.1
Noisy samples: [2]


In [13]:
minority_indices, boundary_indices, majority_indices = get_category_indices(X_new, y_new)
print(f"Minority: {minority_indices}")
print(f"Majority: {majority_indices}")
X_minority = X_new[minority_indices]
y_minority = y_new[minority_indices]

X_majority = X_new[majority_indices]
y_majority = y_new[majority_indices]
sorted_majority = sort_majority_samples(X_majority)

X_resampled = np.append(X_minority, X_majority[:len(X_minority)], axis=0)
y_resampled = np.append(y_minority, y_majority[:len(y_minority)], axis=0)

print(len(X_resampled))
print(X_resampled)

Classes: [0 1]
Minority Class: 1
Minority Ratio: 0.10101010101010101
Minority: [ 3  7  8 13 15 39 66 69 70 73]
Majority: [ 0  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72
 73 74 75 76 77 79 80 81 82 83 84 86 87 88]
20
[[5.27613616 4.69903979 5.25064979 4.44443764 4.80141202 4.08664077
  6.39733293 5.45274039 3.46957154 6.63792819 4.63709185 4.80735966
  4.22904    3.22986944 5.31326774 4.63159882 4.19576324 7.20693164
  3.36069237 3.43058346]
 [5.20246832 2.4735707  3.23364364 3.12269894 4.15981345 4.55199551
  5.9379377  3.05269765 3.11191149 6.9573562  3.49570853 5.12784295
  5.13786391 4.33412154 4.14198972 3.9069939  5.59297464 6.63042809
  4.46013127 3.81534493]
 [2.42237236 4.7585646  4.10638662 4.30747543 3.88072661 4.32848553
  6.4391136  4.9482475  0.         5.47151514 5.37273017 2.95593654
  4.13941455 5.6991